# Read PM2.5 from txt file
### First we gonna looki for some information inside file

In [88]:
def show_text(fileName, n_line):
    f = open(fileName, "r")
    for i in range(n_line):
        print(f.readline())
    f.close()

In [89]:
show_text("PM2.5_kaggle 2021/Bangkok/Train/Bangkok.txt",20)

% Country: Thailand

% City: Bangkok

% City (ASCII): Bangkok

% Region: Bangkok

% Region (ASCII): Bangkok

% Population: 5104476

% Latitude: 13.754

% Longitude: 100.5014

% Time Zone: Asia/Bangkok

% Year, Month, Day, UTC Hour, PM2.5, PM10_mask, Retrospective

2016	3	3	8	62.9	1.00	0

2016	3	3	9	62.9	1.00	0

2016	3	3	10	55.5	1.00	0

2016	3	3	11	55.5	1.00	0

2016	3	3	12	47.9	1.00	0

2016	3	3	13	43.6	1.00	0

2016	3	3	14	28.6	1.00	0

2016	3	3	15	33.6	1.00	0

2016	3	3	16	34.8	1.00	0

2016	3	3	17	31.3	1.00	0



In [90]:
show_text("PM2.5_kaggle 2021/Chanthaburi/Train/Chanthaburi.txt",20)

% Country: Thailand

% City: Chanthaburi

% City (ASCII): Chanthaburi

% Region: Chanthaburi

% Region (ASCII): Chanthaburi

% Population: 99819

% Latitude: 12.6096

% Longitude: 102.1045

% Time Zone: Asia/Bangkok

% Year, Month, Day, UTC Hour, PM2.5, PM10_mask, Retrospective

2016	3	3	8	44.2	1.00	0

2016	3	3	9	44.2	1.00	0

2016	3	3	10	44.3	1.00	0

2016	3	3	11	44.2	1.00	0

2016	3	3	12	34.2	1.00	0

2016	3	3	13	34.6	1.00	0

2016	3	3	14	31.0	1.00	0

2016	3	3	15	31.6	1.00	0

2016	3	3	16	30.1	1.00	0

2016	3	3	17	45.5	1.00	0



# Convert text to Dataframe

In [91]:
import numpy as np
import pandas as pd

In [92]:
def txt_to_df(fileName):

    ### Open file named "fileName"
    f = open(fileName,"r")

    ### count for check line number for decision making
    count = 0

    ### for each line in fileName ###
    for line in f:

        ### tmp represents each line in fileName depends on count
        tmp = line

        ### When count == 9 that is header we will use that for columns name in dataframe
        if count == 9:

            ### We remove "% " at first 2 string and split each string 
            header = tmp[2:].split(", ")

            ### The last word will contains "\n" so we remove it
            header[-1] = header[-1][:-1]

            ### Create dataframe to support data will be coming after this line
            df = pd.DataFrame(columns = header)

        ### After this line will be only content in dataframe
        elif count > 9:

            ### Split file as list
            data = tmp.split()

            ### Create data as dataframe
            data_df = pd.DataFrame([data], columns = header)

            ### Append data each line to dataframe we gonna use it for output
            df = df.append(data_df)

        ### Increment count for known line number
        count += 1
    f.close()
    
    df = df.reset_index().drop(columns = ["index"])
    return df

In [93]:
df = txt_to_df("PM2.5_kaggle 2021/Bangkok/Train/Bangkok.txt")

In [94]:
df

,Year,Month,Day,UTC Hour,PM2.5,PM10_mask,Retrospective
0,2016,3,3,8,62.9,1.00,0
1,2016,3,3,9,62.9,1.00,0
2,2016,3,3,10,55.5,1.00,0
3,2016,3,3,11,55.5,1.00,0
4,2016,3,3,12,47.9,1.00,0
...,...,...,...,...,...,...,...
24388,2019,3,17,19,42.2,0.00,0
24389,2019,3,17,20,41.2,0.00,0
24390,2019,3,17,21,37.7,0.00,0
24391,2019,3,17,22,39.0,0.00,0


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24393 entries, 0 to 24392
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           24393 non-null  object
 1   Month          24393 non-null  object
 2   Day            24393 non-null  object
 3   UTC Hour       24393 non-null  object
 4   PM2.5          24393 non-null  object
 5   PM10_mask      24393 non-null  object
 6   Retrospective  24393 non-null  object
dtypes: object(7)
memory usage: 1.3+ MB


## Change value each header to numeric

In [96]:
header_to_numeric = ["PM2.5", "PM10_mask", "Retrospective"]
for header in header_to_numeric:
    df[header] = pd.to_numeric(df[header])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24393 entries, 0 to 24392
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           24393 non-null  object 
 1   Month          24393 non-null  object 
 2   Day            24393 non-null  object 
 3   UTC Hour       24393 non-null  object 
 4   PM2.5          24393 non-null  float64
 5   PM10_mask      24393 non-null  float64
 6   Retrospective  24393 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 1.3+ MB


## Convert revelant date columns to datetime

In [105]:
from datetime import datetime, timedelta

In [103]:
def make_len2(time):
    if len(time) == 1:
        time = "0" + time
    return time

def all_make_len2(df):
    df["Month"] = df["Month"].apply(lambda x: make_len2(x))
    df["Day"] = df["Day"].apply(lambda x: make_len2(x))
    df["UTC Hour"] = df["UTC Hour"].apply(lambda x: make_len2(x))
    return df

def create_datetime(df):
    df["datestr"] = df["Year"]+df["Month"]+df["Day"]+df["UTC Hour"]
    df["datetime"] = pd.to_datetime(df["datestr"], format="%Y%m%d%H")
    return df

In [106]:
df = all_make_len2(df)
df = create_datetime(df)
df["datetime"] = df["datetime"] + timedelta(hours=7)
df

,Year,Month,Day,UTC Hour,PM2.5,PM10_mask,Retrospective,datestr,datetime
0,2016,03,03,08,62.9,1.0,0,2016030308,2016-03-03 15:00:00
1,2016,03,03,09,62.9,1.0,0,2016030309,2016-03-03 16:00:00
2,2016,03,03,10,55.5,1.0,0,2016030310,2016-03-03 17:00:00
3,2016,03,03,11,55.5,1.0,0,2016030311,2016-03-03 18:00:00
4,2016,03,03,12,47.9,1.0,0,2016030312,2016-03-03 19:00:00
...,...,...,...,...,...,...,...,...,...
24388,2019,03,17,19,42.2,0.0,0,2019031719,2019-03-18 02:00:00
24389,2019,03,17,20,41.2,0.0,0,2019031720,2019-03-18 03:00:00
24390,2019,03,17,21,37.7,0.0,0,2019031721,2019-03-18 04:00:00
24391,2019,03,17,22,39.0,0.0,0,2019031722,2019-03-18 05:00:00


## Convert raw to dataframe with correct utc by fileName only

In [107]:
def raw_txt_to_correct_df(fileName):

    ########################################################################################
    # This function is use for convert raw PM2.5 data from kaggle to dataframe with UTC +7 #
    ########################################################################################
    
    df = txt_to_df(fileName)
    header_to_numeric = ["PM2.5", "PM10_mask", "Retrospective"]
    for header in header_to_numeric:
        df[header] = pd.to_numeric(df[header])
    df = all_make_len2(df)
    df = create_datetime(df)
    df["datetime"] = df["datetime"] + timedelta(hours=7)
    return df

In [109]:
df = raw_txt_to_correct_df("PM2.5_kaggle 2021/Bangkok/Train/Bangkok.txt")
df

,Year,Month,Day,UTC Hour,PM2.5,PM10_mask,Retrospective,datestr,datetime
0,2016,03,03,08,62.9,1.0,0,2016030308,2016-03-03 15:00:00
1,2016,03,03,09,62.9,1.0,0,2016030309,2016-03-03 16:00:00
2,2016,03,03,10,55.5,1.0,0,2016030310,2016-03-03 17:00:00
3,2016,03,03,11,55.5,1.0,0,2016030311,2016-03-03 18:00:00
4,2016,03,03,12,47.9,1.0,0,2016030312,2016-03-03 19:00:00
...,...,...,...,...,...,...,...,...,...
24388,2019,03,17,19,42.2,0.0,0,2019031719,2019-03-18 02:00:00
24389,2019,03,17,20,41.2,0.0,0,2019031720,2019-03-18 03:00:00
24390,2019,03,17,21,37.7,0.0,0,2019031721,2019-03-18 04:00:00
24391,2019,03,17,22,39.0,0.0,0,2019031722,2019-03-18 05:00:00


# Wind

In [11]:
pd.read_csv("PM2.5_kaggle 2021/Bangkok/Train/3H_wind_Bangkok.csv").head(20)

,datetime,WindDir,Wind Speed(km/h),lat,long
0,2016-03-03 01:00:00,45,13,13.754,100.5014
1,2016-03-03 04:00:00,60,11,13.754,100.5014
2,2016-03-03 07:00:00,65,15,13.754,100.5014
3,2016-03-03 10:00:00,75,13,13.754,100.5014
4,2016-03-03 13:00:00,70,12,13.754,100.5014
5,2016-03-03 16:00:00,60,12,13.754,100.5014
6,2016-03-03 19:00:00,70,11,13.754,100.5014
7,2016-03-03 22:00:00,70,15,13.754,100.5014
8,2016-03-04 01:00:00,95,13,13.754,100.5014
9,2016-03-04 04:00:00,115,11,13.754,100.5014


# Temperature

In [10]:
pd.read_csv("PM2.5_kaggle 2021/Bangkok/Train/3H_temperature_Bangkok.csv").head(20)

,datetime,Temp(C),lat,long
0,2016-03-03 01:00:00,25.5,13.754,100.5014
1,2016-03-03 04:00:00,25.0,13.754,100.5014
2,2016-03-03 07:00:00,26.4,13.754,100.5014
3,2016-03-03 10:00:00,31.4,13.754,100.5014
4,2016-03-03 13:00:00,34.1,13.754,100.5014
5,2016-03-03 16:00:00,34.6,13.754,100.5014
6,2016-03-03 19:00:00,29.1,13.754,100.5014
7,2016-03-03 22:00:00,27.3,13.754,100.5014
8,2016-03-04 01:00:00,26.5,13.754,100.5014
9,2016-03-04 04:00:00,25.8,13.754,100.5014


# Hotspot

In [15]:
pd.read_csv("PM2.5_kaggle 2021/Fire hotspot/fire_archive_M6_163549.csv")

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-0.8235,133.0450,302.8,1.1,1.0,2019-12-01,130,Terra,MODIS,33,6.03,280.6,2.5,D,0
1,-0.8162,133.0429,312.8,1.1,1.0,2019-12-01,130,Terra,MODIS,65,6.03,285.2,8.8,D,0
2,-12.3706,131.5872,329.4,1.0,1.0,2019-12-01,133,Terra,MODIS,30,6.03,298.1,14.8,D,0
3,-12.3665,131.5595,331.9,1.0,1.0,2019-12-01,133,Terra,MODIS,58,6.03,305.5,16.9,D,0
4,-12.3651,131.5503,331.6,1.0,1.0,2019-12-01,133,Terra,MODIS,48,6.03,304.9,16.0,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372769,40.6052,109.6865,304.4,1.0,1.0,2020-07-31,1847,Aqua,MODIS,60,6.03,290.9,6.3,N,2
372770,39.4083,110.6341,300.0,1.1,1.0,2020-07-31,1847,Aqua,MODIS,5,6.03,284.1,5.1,N,0
372771,39.3870,110.5909,300.1,1.1,1.0,2020-07-31,1847,Aqua,MODIS,15,6.03,285.9,4.9,N,0
372772,44.7340,90.2537,304.1,1.3,1.1,2020-07-31,2025,Aqua,MODIS,58,6.03,287.6,11.0,N,2


# Submission Example

In [13]:
pd.read_excel("PM2.5_kaggle 2021/submission_example_Chanthaburi_3days.xlsx").head(20)

,station,index,Predicted,PM2.5
0,Chanthaburi,2019-03-18 12:00:00,2019-03-18 13:00:00,33.5
1,NaN,NaT,2019-03-18 14:00:00,33.5
2,NaN,NaT,2019-03-18 15:00:00,35.2
3,NaN,NaT,2019-03-18 16:00:00,32.9
4,NaN,NaT,2019-03-18 17:00:00,30.5
5,NaN,NaT,2019-03-18 18:00:00,31.4
6,NaN,NaT,2019-03-18 19:00:00,32.8
7,NaN,NaT,2019-03-18 20:00:00,32.2
8,NaN,NaT,2019-03-18 21:00:00,30.7
9,NaN,NaT,2019-03-18 22:00:00,32.1


In [14]:
pd.read_csv("PM2.5_kaggle 2021/submission_example_Kaggle.csv").head(20)

,Id,Expected
0,1,34.71
1,2,33.97
2,3,36.97
3,4,33.82
4,5,32.14
5,6,32.35
6,7,33.33
7,8,33.39
8,9,32.05
9,10,33.36
